In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import json

In [2]:
with open( '../Data/VAF_Data/p377_variant_summary.html', 'r') as mf:
    file = mf.read()

In [3]:
soup = BeautifulSoup(file, 'html.parser')

In [4]:
mysoup = soup.body.div.find_all("script")

In [5]:
mysoup = soup.body.div.find_all("script")[1].contents[0].strip()
mysoup

'{"columns":[{"label":["TARGET_NAME"],"name":[1],"type":["fctr"],"align":["left"]},{"label":["PATIENT"],"name":[2],"type":["fctr"],"align":["left"]},{"label":["CHR"],"name":[3],"type":["fctr"],"align":["left"]},{"label":["START"],"name":[4],"type":["int"],"align":["right"]},{"label":["END"],"name":[5],"type":["int"],"align":["right"]},{"label":["REF"],"name":[6],"type":["fctr"],"align":["left"]},{"label":["ALT"],"name":[7],"type":["fctr"],"align":["left"]},{"label":["GENE"],"name":[8],"type":["fctr"],"align":["left"]},{"label":["PROTEIN_CHANGE"],"name":[9],"type":["fctr"],"align":["left"]},{"label":["cDNA_CHANGE"],"name":[10],"type":["fctr"],"align":["left"]},{"label":["TARGET_VAF_MEAN"],"name":[11],"type":["dbl"],"align":["right"]},{"label":["Autotate_annot1"],"name":[12],"type":["fctr"],"align":["left"]},{"label":["Autotate_annot2"],"name":[13],"type":["fctr"],"align":["left"]},{"label":["VARIANT"],"name":[14],"type":["chr"],"align":["left"]},{"label":["sampleID_short"],"name":[15],"

In [6]:
table_dict = json.loads(mysoup)

for key in table_dict:
    print(key)

columns
data
options


In [7]:
columns = [stuff["label"][0] for stuff in table_dict["columns"]]

In [8]:
table_dict["data"]

data = []
for stuff in table_dict["data"]:
    rows = []
    for key in stuff:
        rows.append(stuff[key])
    data.append(rows)
    
variant_annotations = pd.DataFrame(data, columns = columns)
variant_annotations

,TARGET_NAME,PATIENT,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,cDNA_CHANGE,TARGET_VAF_MEAN,Autotate_annot1,Autotate_annot2,VARIANT,sampleID_short
0,IID_H198302_T01_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.410,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T01_01_TD01
1,IID_H198302_T02_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.395,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T02_01_TD01
2,IID_H198302_T03_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.437,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T03_01_TD01
3,IID_H198302_T04_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.477,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T04_01_TD01
4,IID_H198302_T01_01_TD01,IID_H198302,4,106157152,106157152,C,T,TET2,p.Q685*,c.2053C>T,0.474,AUTO_OK,ONCOGENIC,TET2_p.Q685*,T01_01_TD01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,IID_H198303_T02_01_TD01,IID_H198303,4,106164802,106164803,GC,G,TET2,p.A1224fs*2,c.3671delC,0.455,AUTO_OK,ONCOGENIC,TET2_p.A1224fs*2,T02_01_TD01
71,IID_H198303_T03_01_TD01,IID_H198303,4,106164802,106164803,GC,G,TET2,p.A1224fs*2,c.3671delC,0.482,AUTO_OK,ONCOGENIC,TET2_p.A1224fs*2,T03_01_TD01
72,IID_H198304_T01_01_TD01,IID_H198304,4,106164782,106164783,CT,C,TET2,p.G1218fs*8,c.3651delT,0.332,AUTO_OK,ONCOGENIC,TET2_p.G1218fs*8,T01_01_TD01
73,IID_H198304_T02_01_TD01,IID_H198304,4,106164782,106164783,CT,C,TET2,p.G1218fs*8,c.3651delT,0.374,AUTO_OK,ONCOGENIC,TET2_p.G1218fs*8,T02_01_TD01


In [9]:
#save supplied data
variant_annotations.to_csv('../Data/VAF_Data/MSK_VAFS.tsv', sep = '\t')

In [10]:
#Add in patient initials
variant_annotations['pat'] = variant_annotations['PATIENT']

init_dict = {
    'IID_H198302': 'EL',
    'IID_H198303': 'GR',
    'IID_H198304': 'RN'
}

variant_annotations['patient_init'] = variant_annotations['pat'].replace(init_dict)

In [11]:
#Add in cell type
variant_annotations['cel'] = variant_annotations['TARGET_NAME']
ct_dict = {
    'T01': 'BM',
    'T02': 'Mono',
    'T03': 'nBC',
    'T04': 'NE'
}

patients = variant_annotations['TARGET_NAME'].to_list()
cells = []
for pat in patients: 
    for a in ct_dict:
        #print(a, pat)
        x = (pat.split('_')[2])
        if a == x:
            cells.append(ct_dict[a])
            
cell_dict = dict(zip(patients, cells))   

variant_annotations['celltype'] = variant_annotations['cel'].replace(cell_dict)

variant_annotations = variant_annotations.drop(columns = ['cel', 'pat'])
variant_annotations

,TARGET_NAME,PATIENT,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,cDNA_CHANGE,TARGET_VAF_MEAN,Autotate_annot1,Autotate_annot2,VARIANT,sampleID_short,patient_init,celltype
0,IID_H198302_T01_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.410,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T01_01_TD01,EL,BM
1,IID_H198302_T02_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.395,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T02_01_TD01,EL,Mono
2,IID_H198302_T03_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.437,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T03_01_TD01,EL,nBC
3,IID_H198302_T04_01_TD01,IID_H198302,1,33134916,33134916,T,A,RBBP4,p.N282K,c.846T>A,0.477,AUTO_OK,UNKNOWN,RBBP4_p.N282K,T04_01_TD01,EL,NE
4,IID_H198302_T01_01_TD01,IID_H198302,4,106157152,106157152,C,T,TET2,p.Q685*,c.2053C>T,0.474,AUTO_OK,ONCOGENIC,TET2_p.Q685*,T01_01_TD01,EL,BM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,IID_H198303_T02_01_TD01,IID_H198303,4,106164802,106164803,GC,G,TET2,p.A1224fs*2,c.3671delC,0.455,AUTO_OK,ONCOGENIC,TET2_p.A1224fs*2,T02_01_TD01,GR,Mono
71,IID_H198303_T03_01_TD01,IID_H198303,4,106164802,106164803,GC,G,TET2,p.A1224fs*2,c.3671delC,0.482,AUTO_OK,ONCOGENIC,TET2_p.A1224fs*2,T03_01_TD01,GR,nBC
72,IID_H198304_T01_01_TD01,IID_H198304,4,106164782,106164783,CT,C,TET2,p.G1218fs*8,c.3651delT,0.332,AUTO_OK,ONCOGENIC,TET2_p.G1218fs*8,T01_01_TD01,RN,BM
73,IID_H198304_T02_01_TD01,IID_H198304,4,106164782,106164783,CT,C,TET2,p.G1218fs*8,c.3651delT,0.374,AUTO_OK,ONCOGENIC,TET2_p.G1218fs*8,T02_01_TD01,RN,Mono


In [12]:
#save annotated data
variant_annotations.to_csv('../Data/VAF_Data/MSK_VAFS_PT_CELL.tsv', sep = '\t')